In [20]:
import requests

In [21]:
import geopandas as gpd
import numpy as np
import pandas as pd

## Open State data

In [22]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
state_gdf = gpd.read_file(file_path)

## Get Columm Data

In [23]:
table = "B20002"

url = "https://api.census.gov/data/2023/acs/acs5"
params = {"get": f"group({table})", "for": "state:*"}  # <-- all states
response = requests.get(url, params=params)

In [24]:
data = response.json()
columns = data[0]
rows = data[1:]
income_df = pd.DataFrame(rows, columns=columns)

In [25]:
url = "https://api.census.gov/data/2023/acs/acs5/variables.json"
response = requests.get(url)
variables = response.json()["variables"]

In [26]:
col_vars = {
    k: v["label"].split("!!")[-1] for k, v in variables.items() if k.startswith(table)
}
col_vars["GEO_ID"] = "AFFGEOID"

In [27]:
income_df = income_df.rename(columns=col_vars)
income_df = income_df[list(col_vars.values())]

In [28]:
income_df[["Total:", "Male", "Female"]] = income_df[
    ["Total:", "Male", "Female"]
].astype(int)
income_df = income_df.replace(-666666666, np.nan)

In [29]:
income_df["DIFF"] = income_df["Male"] - income_df["Female"]

## Merge Data

In [32]:
income_gdf = state_gdf.merge(income_df, on="AFFGEOID", how="inner")

In [33]:
income_gdf = income_gdf.to_crs(9311)
income_gdf.to_file("data/Median_Income_Different_Male_Female_Per_State.gpkg")